In [4]:
from nba_api.stats.endpoints import teamgamelogs
from nba_api.stats.endpoints import boxscoreadvancedv2
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguedashteamstats
import pandas as pd
import matplotlib.pyplot as plt

# Define the Team ID for the Orlando Magic
team_id = 1610612753
season = "2023-24"
last_n_games = 5
location = ""
season_type = "Regular Season"

team_gamelogs = teamgamelogs.TeamGameLogs(
    team_id_nullable=team_id,
    season_nullable=season,
    last_n_games_nullable=last_n_games,
    location_nullable=location,
    season_type_nullable=season_type,
)
team_gamelogs_df = team_gamelogs.get_data_frames()[0]

# use boxscoreadvancedv2 to fetch the advanced boxscores for each game and save the team advanced stats to a dataframe
team_advanced_stats = []
for game_id in team_gamelogs_df["GAME_ID"]:
    try:
        # get the date of the game in format MM/DD removing T:00:00:00 from the end
        date = team_gamelogs_df.loc[
            team_gamelogs_df["GAME_ID"] == game_id, "GAME_DATE"
        ].values[0][5:-9]
        # get the opponent name
        opponent = team_gamelogs_df.loc[
            team_gamelogs_df["GAME_ID"] == game_id, "MATCHUP"
        ].values[0][-3:]
        boxscore_advanced = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game_id)
        boxscore_advanced_df = boxscore_advanced.get_data_frames()[0]
        # select the desired team advanced stats columns from the dataframe
        team_advanced_stats.append(
            boxscore_advanced_df.loc[0, ["OFF_RATING", "DEF_RATING"]]
        )
        team_advanced_stats[-1]["GAME_ID"] = f"{date} {opponent}"
    except Exception as e:
        print(f"Error fetching data for game ID {game_id}: {str(e)}")

# convert the list of team advanced stats to a dataframe
team_advanced_stats_df = pd.concat(team_advanced_stats)

print(team_advanced_stats_df)

# plot the offensive and defensive ratings for the last 5 games, the team average and the league average
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(
    team_advanced_stats_df["GAME_ID"],
    team_advanced_stats_df["OFF_RATING"],
    color="red",
    label="Offensive Rating",
)
ax.plot(
    team_advanced_stats_df["GAME_ID"],
    team_advanced_stats_df["DEF_RATING"],
    color="blue",
    label="Defensive Rating",
)
ax.axhline(
    team_average_offensive_rating,
    color="red",
    linestyle="--",
    label="Team Average Offensive Rating",
)
ax.axhline(
    team_average_defensive_rating,
    color="blue",
    linestyle="--",
    label="Team Average Defensive Rating",
)
ax.axhline(
    league_average_offensive_rating,
    color="red",
    linestyle=":",
    label="League Average Offensive Rating",
)
ax.axhline(
    league_average_defensive_rating,
    color="blue",
    linestyle=":",
    label="League Average Defensive Rating",
)
ax.set_title(
    f"Last {last_n_games} Games Offensive and Defensive Ratings for the Orlando Magic"
)
ax.set_xlabel("Game")
ax.set_ylabel("Rating")
ax.legend()
plt.show()

KeyError: 'OFF_RATING'